In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


Referenced the code from :
</br>
https://toarches.medium.com/image-video-and-real-time-webcam-object-detection-and-instance-segmentation-with-mask-r-cnn-37a4675dcb49

In [2]:
!git clone https://github.com/matterport/Mask_RCNN.git

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 125.23 MiB | 33.71 MiB/s, done.
Resolving deltas: 100% (562/562), done.


In [3]:
!pip install pytube

     |████████████████████████████████| 51kB 4.5MB/s 


In [5]:
#Download video from YouTube
import pytube
from pytube import YouTube
import os

def downloadYouTube(videourl, path):
    yt = YouTube(videourl)
    yt = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    if not os.path.exists(path):
        os.makedirs(path)
        yt.download(path)

downloadYouTube('https://www.youtube.com/watch?v=wdfiyArulLc&ab_channel=HoonYoungYoon', './samples/videos')

In [4]:
# import libraries
import os, sys
import sys
import random
import math
import numpy as np
import scipy.misc
import matplotlib
import matplotlib.pyplot as plt


ROOT_DIR = os.path.abspath("../content/Mask_RCNN/")
sys.path.append(ROOT_DIR)

MODEL_DIR=os.path.join(ROOT_DIR, "logs/")

from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))
import coco

# define random colors
def random_colors(N):
  np.random.seed(1)
  colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
  return colors

#apply mask to image
def apply_mask(image, mask, color, alpha=0.5):
  for n, c in enumerate(color):
    image[:, :, n] = np.where(
        mask == 1,
        image[:,:,n]*(1-alpha)+alpha*c,
        image[:,:,n]
        )
    return image

#take the image and apply the mask, box, and Label
def display_instances(image, boxes, masks, ids, names, scores):
  n_instances = boxes.shape[0]
  colors = random_colors(n_instances)
  
  if not n_instances:
    print('NO INSTANCES TO DISPLAY')
  else:
    assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]
    
  for i, color in enumerate(colors):
    if not np.any(boxes[i]):
      continue
    y1, x1, y2, x2 = boxes[i]
    label = names[ids[i]]
    score = scores[i] if scores is not None else None
    caption = '{} {:.2f}'.format(label, score) if score else label
    mask = masks[:, :, i]
  
    image = apply_mask(image, mask, color)
    image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
    image = cv2.putText(
        image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
        )
  return image

Using TensorFlow backend.


In [6]:
#Making Directories
ROOT_DIR = os.path.abspath("../content/")

VIDEO_DIR =  os.path.abspath("../content/samples/videos/")
if os.path.isdir(VIDEO_DIR)==False:
  os.mkdir(VIDEO_DIR)
  print('VIDEO_DIR was not existed; created new folder for VIDEO_DIR')

VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "savedimgs/")
if os.path.isdir(VIDEO_SAVE_DIR)==False:
  os.mkdir(VIDEO_SAVE_DIR)
  print('VIDEO_SAVE_DIR was not existed; created new folder for VIDEO_SAVE_DIR')

COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
if not os.path.exists(COCO_MODEL_PATH):
  utils.download_trained_weights(COCO_MODEL_PATH)

MODEL_DIR=os.path.abspath("../content/Mask_RCNN/logs/")
if os.path.isdir(MODEL_DIR)==False:
  os.mkdir(MODEL_DIR)
  print('log folder for MODEL_DIR was not existed; created new folder for MODEL_DIR')

VIDEO_SAVE_DIR was not existed; created new folder for VIDEO_SAVE_DIR
... done downloading pretrained model!
log folder for MODEL_DIR was not existed; created new folder for MODEL_DIR


In [7]:
batch_size = 3

class InferenceConfig(coco.CocoConfig):
  GPU_COUNT = 1
  IMAGES_PER_GPU = batch_size

config = InferenceConfig()
print(config)

# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [9]:
import cv2
video = cv2.VideoCapture(os.path.join(VIDEO_DIR, 'IMG_0125.mp4'))
# Find OpenCV version
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

if int(major_ver) < 3 :
  fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
  print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
else :
  fps = video.get(cv2.CAP_PROP_FPS)
  print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))


frames =[]
frame_count=0

while True:
  ret, frame = video.read() 
  if not ret:
    break
  
  frame_count+=1
  frames.append(frame)
  print('frame_count : {0}'.format(frame_count))
  
  if len(frames)==batch_size:
    results = model.detect(frames, verbose=0)
  
    for index, item in enumerate(zip(frames, results)):
      frame = item[0]
      r=item[1]
      frame=display_instances(frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])
      name = '{0}.jpg'.format(frame_count + index - batch_size)
      name=os.path.join(VIDEO_SAVE_DIR, name)
      cv2.imwrite(name, frame)
    frames=[]

video.release()

Frames per second using video.get(cv2.CAP_PROP_FPS) : 30.0
frame_count : 1
frame_count : 2
frame_count : 3

frame_count : 4
frame_count : 5
frame_count : 6
frame_count : 7
frame_count : 8
frame_count : 9
frame_count : 10
frame_count : 11
frame_count : 12
frame_count : 13
frame_count : 14
frame_count : 15
frame_count : 16
frame_count : 17
frame_count : 18
frame_count : 19
frame_count : 20
frame_count : 21
frame_count : 22
frame_count : 23
frame_count : 24
frame_count : 25
frame_count : 26
frame_count : 27
frame_count : 28
frame_count : 29
frame_count : 30
frame_count : 31
frame_count : 32
frame_count : 33
frame_count : 34
frame_count : 35
frame_count : 36
frame_count : 37
frame_count : 38
frame_count : 39
frame_count : 40
frame_count : 41
frame_count : 42
frame_count : 43
frame_count : 44
frame_count : 45
frame_count : 46
frame_count : 47
frame_count : 48
frame_count : 49
frame_count : 50
frame_count : 51
frame_count : 52
frame_count : 53
frame_count : 54
frame_count : 55
frame_count : 

In [10]:
#Create a video from a list of segmented images.
import glob
import os

def make_video(outvid, images=None, fps=30, size=None, is_color=True, format="FMP4"):
  from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
  fourcc = VideoWriter_fourcc(*format)
  vid = None

  for image in images:
    if not os.path.exists(image):
      raise FileNotFoundError(image)
    img = imread(image)
    if vid is None:
      if size is None:
        size = img.shape[1], img.shape[0]
      vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
    
    if size[0] != img.shape[1] and size[1] != img.shape[0]:
      img = resize(img, size)
    vid.write(img)
  vid.release()
  return vid

# Path Configuration
VIDEO_DIR =  os.path.abspath("../content/samples/videos/")
VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "savedimgs")
images = list(glob.iglob(os.path.join(VIDEO_SAVE_DIR, '*.*')))
# Sort the images by integer index
images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))

outvid = os.path.join(VIDEO_DIR, "IMG_0125_processed.mp4")
make_video(outvid, images, fps=30)

<VideoWriter 0x7f6c32c4af10>